In [34]:
# We import all the necessary packages
import os
import re
import collections
import matplotlib.pyplot as plt
import csv
from textblob import TextBlob
import pandas as pd
import sklearn
# import Pickle
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import learning_curve

from sklearn.ensemble import RandomForestClassifier
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
path ="/content/drive/MyDrive/"

In [36]:
train = pd.read_csv("train.csv")
val = pd.read_csv("validation.csv")
test = pd.read_csv("test.csv")

X_train, y_train = train["text"], train["spam"]
X_val, y_val = val["text"], val["spam"]
X_test, y_test = test["text"], test["spam"]

In [37]:
bow_transformer = CountVectorizer().fit(train['text'])

In [38]:
emails_bow = bow_transformer.transform(train['text'])

In [39]:
 tfidf_transformer = TfidfTransformer().fit(emails_bow)

MODEL:1 Naive Bayes Classifier

In [40]:

# Build and train the model
model = Pipeline([('bow',CountVectorizer()),('tfidf', TfidfTransformer()),('classifier', MultinomialNB())])

In [41]:
%time model_nb=model.fit(X_train,y_train)

CPU times: user 587 ms, sys: 6.42 ms, total: 593 ms
Wall time: 601 ms


In [42]:
# Make predictions on the test set
predictions = model_nb.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.2f}")

# Display classification report
print(classification_report(y_test, predictions))

Accuracy: 0.88
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       856
           1       1.00      0.54      0.70       290

    accuracy                           0.88      1146
   macro avg       0.93      0.77      0.82      1146
weighted avg       0.90      0.88      0.87      1146



MODEL 2: Support Vector Machine

In [43]:

pipeline_sv = Pipeline([
    ('bow', CountVectorizer()),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', SVC()),  # train on TF-IDF vectors w/ SVM classifier
])

In [44]:
%time model_sv = pipeline_sv.fit(X_train, y_train)

CPU times: user 6.69 s, sys: 61.9 ms, total: 6.75 s
Wall time: 6.78 s


In [45]:
print(model_sv.score(X_train, y_train))
print(model_sv.score(X_val, y_val))

1.0
0.9836423118865867


In [46]:

predictions = model_sv.predict(X_test)
print (confusion_matrix(y_test, predictions))
print (classification_report(y_test, predictions))

[[854   2]
 [ 12 278]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       856
           1       0.99      0.96      0.98       290

    accuracy                           0.99      1146
   macro avg       0.99      0.98      0.98      1146
weighted avg       0.99      0.99      0.99      1146



MODEL 3: Random Forest Classifier

In [47]:
from sklearn.ensemble import RandomForestClassifier
pipeline_rf = Pipeline([
    ('bow', CountVectorizer()),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', RandomForestClassifier()),  # train on TF-IDF vectors w/ SVM classifier
])

In [48]:
%time model_rf = pipeline_rf.fit(X_train, y_train)

CPU times: user 3.77 s, sys: 11 ms, total: 3.78 s
Wall time: 3.87 s


In [49]:
print(model_rf.score(X_train, y_train))
print(model_rf.score(X_val, y_val))

1.0
0.9694656488549618


In [50]:
predictions = model_rf.predict(X_test)
print (confusion_matrix(y_test, predictions))
print (classification_report(y_test, predictions))

[[855   1]
 [ 37 253]]
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       856
           1       1.00      0.87      0.93       290

    accuracy                           0.97      1146
   macro avg       0.98      0.94      0.95      1146
weighted avg       0.97      0.97      0.97      1146



we observe that the SVM gives the best accuracy amongst the three classifiers.